# Extract results of activation computations for UPP08 to RWCL

Dmitry Portnov, Sep 2021

## Goals and input data

Extract data from "component_to_cell.json" and form a table of values corresponding to RWCL "Radwaste_Checklist_UPP08.xlsx" and "Radwaste_Checklist_UP08_ISS_PCSS.xlsx"
The component to cell mapping "component-to-cell-4.json" is used.

Al the files are in the same folder on 10.106.203.11: "d:\dvp\dev\mcnp\upp\wrk\up08".


## Scenario

- Load data from json as map RWCL item -> list of cells in the MCNP model
- Load activation data as map cell -> activation values for 12 day of cooliing



In [1]:
%config Completer.use_jedi = False

In [2]:
import typing as tp
from typing import Dict

In [3]:
import os, sys

In [4]:
try:
    import matplotlib as mpl
except ImportError:
    !pip install matplotlib
    import matplotlib as mpl

import matplotlib.pyplot as plt 
%matplotlib inline

In [5]:
import json
import pathlib

from collections import defaultdict
from pathlib import Path
from pprint import pprint, pformat


In [6]:
import numpy as np
import dotenv
from tqdm import tqdm

In [7]:
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

In [8]:
try:
    import openpyxl
except ImportError:
    !pip install openpyxl
    import openpyxl

In [9]:
import mckit as mc
import r2s_rfda as r2s
import r2s_rfda.fetch as r2sft
import r2s_rfda.utils as r2sut
import r2s_rfda.launcher as r2sln

# from r2s_rfda.fetch import load_result_config, load_data, load_result_config
# from r2s_rfda import utils
# from r2s_rfda.launcher import load_config


---*** D:\programs\anaconda3\envs\r2s-0.3\lib\site-packages\mckit\nlopt.dll


In [10]:
# If exists local ".env" file, then load it to environment.
dotenv.load_dotenv();

In [11]:
def all_exist(*paths: Path) -> bool:
    for p in paths: # type: Path
        if not p.exists():
            raise FileNotFoundError(p)
def all_are_files(*paths: Path) -> bool:
    return all(map(Path.is_file, paths))
def all_are_dirs(*paths: Path) -> bool:
    return all(map(Path.is_dir, paths))


In [12]:
ROOT: Path = Path(os.getenv("UP08_ACT_ROOT", r'd:\dvp\dev\mcnp\upp\wrk\up08'))
r2swrk: Path = ROOT / "r2s"
component_to_cell_json: Path = ROOT / "component-to-cell-4.json"
rwclinput: Path = ROOT / "Radwaste_Checklist_UPP08.xlsx"
rwclinput_iss: Path = ROOT / "Radwaste_Checklist_UP08_ISS_PCSS.xlsx"
inp_keys_json = ROOT / "inp-keys.json"
all_exist(ROOT, r2swrk, component_to_cell_json, rwclinput, rwclinput_iss, inp_keys_json)
assert all_are_dirs(ROOT, r2swrk)
assert all_are_files(component_to_cell_json, rwclinput, rwclinput_iss, inp_keys_json)


## Load RWCL->cells mapping from JSON

In [13]:
with component_to_cell_json.open() as fid:
    component_to_cell = json.load(fid)
pprint(sorted(component_to_cell.keys()))

['Blade2',
 'Blade3',
 'Blade4',
 'Blade5',
 'Blade6',
 'Blade7',
 'Blade8',
 'DFW_left',
 'DFW_right',
 'DMS_front',
 'DMS_rear',
 'DSM',
 'DSM_down',
 'DSM_up',
 'Flange',
 'Flange_ext',
 'Frame1',
 'Frame_bottom',
 'Frame_top',
 'GDC',
 'Interspace_left_block',
 'Interspace_left_frame',
 'Interspace_lower_shielding',
 'Interspace_rear_frame',
 'Interspace_right_block',
 'Interspace_right_frame',
 'Interspace_top_frame',
 'Levi',
 'Mount_bottom',
 'Mount_left',
 'Mount_right',
 'Mount_up',
 'Rear_box',
 'Reflectometer_left',
 'Reflectometer_rear',
 'Reflectometer_right',
 'Shielding1',
 'Shielding10',
 'Shielding2',
 'Shielding3',
 'Shielding4',
 'Shielding5',
 'Shielding6',
 'Shielding7',
 'Shielding8',
 'Shielding9',
 'Shielding_rear_left',
 'Shielding_rear_right',
 'Shielding_segment_9',
 'Trapezoid',
 'Vis_exvessel',
 'Visir_flange',
 'Visir_invessel',
 'tube_left_down',
 'tube_left_up',
 'tube_right_down',
 'tube_right_up']


In [14]:
with inp_keys_json.open(encoding="utf8") as fid:
    inp_keys = json.load(fid)
# pprint(sorted(inp_keys.keys()))
pprint(inp_keys)

{'DFW Left': ['DFW_left', 'Mount_left', 'tube_left_down', 'tube_left_up'],
 'DFW Right': ['DFW_right', 'Mount_right', 'tube_right_down', 'tube_right_up'],
 'DSM': ['DSM', 'DSM_up', 'DSM_down'],
 'DSM Rails (2 pieces)': ['Взять, как во втором'],
 'Frame Blade #1': ['Frame1'],
 'Frame Blade #2': ['Blade2'],
 'Frame Blade #3': ['Blade3'],
 'Frame Blade #4': ['Blade4'],
 'Frame Blade #5': ['Blade5'],
 'Frame Blade #6': ['Blade6'],
 'Frame Blade #7': ['Blade7'],
 'Frame Blade #8': ['Blade8'],
 'Frame Blade #9': ['Взять, как во втором'],
 'Frame Bottom': ['Frame_bottom'],
 'Frame Top': ['Frame_top'],
 'Generic Screws (50 pieces)': [],
 'Manifold': ['Взять, как во втором'],
 'PP Generics (pads, skids, gripping - 8 pieces)': [],
 'PP Tubes': [],
 'PP structure': ['Trapezoid', 'Rear_box', 'Flange', 'Flange_ext'],
 'Rear box shielding (left)': ['Shielding_rear_left'],
 'Rear box shielding (right)': ['Shielding_rear_right'],
 'Rear box shielding plates (8 pieces)': [],
 'Rear frame shielding1': [

## Load Cell->activation data mapping from R2S working folder

In [15]:
r2s_launch_conf = r2sln.load_config(r2swrk)
r2s_result_conf = r2sft.load_result_config(r2swrk)

In [16]:
time = r2sut.convert_time_literal('12d')
time_labels = list(sorted(r2s_result_conf['gamma'].keys()))
time += time_labels[r2s_launch_conf['zero']]
closest_lab = r2sut.find_closest(time, time_labels)
# closest_lab

In [17]:
r2s_result_conf.keys()

dict_keys(['gamma', 'atoms', 'activity'])

In [18]:
activation_paths = r2s_result_conf["activity"]
activation_paths
act_data = r2sft.load_data(r2swrk / activation_paths[closest_lab])

In [19]:
# Estimate correction factor for tqdm progressbar total value.
its=80625951000
its/act_data._data.nnz

6349.120992777404

In [20]:
tqdm_correction = 6400

In [21]:
def load_cell_activity(cell_excel_path: Path):
    if cell_excel_path.exists():
        cell_activity_df = pd.read_excel(cell_excel_path, sheet_name="activity", index_col=0)
    else:
        cell_activity = defaultdict(float)
        with tqdm(total=act_data._data.nnz * tqdm_correction) as pbar:
            for cnt, ((g, c, i, j, k), act) in enumerate(act_data.iter_nonzero()):
                cell_activity[c] += act
                #print(act_data.gbins[g], c, i, j, k, act)
                #i += 1
                #if i == 1000:
                #    break
                if cnt % 1000 == 0:
                    pbar.update(cnt)
        cell_excel_path.parent.mkdir(parents=True, exist_ok=True)
        index = np.array(sorted(cell_activity.keys()), dtype=np.int32)
        data  = np.fromiter((cell_activity[k] for k in index), dtype=float)
        cell_activity_df = pd.DataFrame(data, columns=["activity"], index=index)
        with pd.ExcelWriter(cell_excel_path, engine="openpyxl") as excel:
            cell_activity_df.to_excel(excel, sheet_name="activity")
    return cell_activity_df
    

In [22]:
def load_cell_nuclide_activity(cell_excel_path: Path, act_data, nuclide: str):
    label = nuclide + " activity"
    loaded_from_cache = False
    if cell_excel_path.exists():
        try:
            cell_activity_df = pd.read_excel(cell_excel_path, sheet_name=label, index_col=0)
            loaded_from_cache = True
        except ValueError:
            loaded_from_cache = False
    
    if not loaded_from_cache:
        nuclide_idx = np.searchsorted(act_data.gbins, nuclide)
        assert act_data.gbins[nuclide_idx] == nuclide
        cell_activity = defaultdict(float)
        with tqdm(total=act_data._data.nnz * tqdm_correction) as pbar:
            for cnt, ((g, c, i, j, k), act) in enumerate(act_data.iter_nonzero()):
                if nuclide_idx == g:
                    cell_activity[c] += act
                if cnt % 1000 == 0:
                    pbar.update(cnt)
        cell_excel_path.parent.mkdir(parents=True, exist_ok=True)
        index = np.array(sorted(cell_activity.keys()), dtype=np.int32)
        data  = np.fromiter((cell_activity[k] for k in index), dtype=float)
        cell_activity_df = pd.DataFrame(data, columns=["activity"], index=index)
        with pd.ExcelWriter(cell_excel_path, engine="openpyxl", mode="w") as excel:
            cell_activity_df.to_excel(excel, sheet_name=label)
    return cell_activity_df
    

In [23]:
cell_activity_df = load_cell_activity(Path.cwd().parent.parent / "wrk/up08/cell_data.xlsx")
cell_activity_df

,activity
320000,4.721641e+13
320001,1.122701e+08
320002,9.906696e+08
320003,4.573004e+07
320004,9.968566e+08
...,...
323774,7.584694e+05
323775,6.154785e+05
323776,5.957511e+06
323777,5.307759e+06


In [24]:
cell_h3_activity_df = load_cell_nuclide_activity(Path.cwd().parent.parent / "wrk/up08/cell_h3_activity.xlsx", act_data, "H3")
cell_h3_activity_df

,activity
320000,1.077859e+09
320001,5.422914e+01
320002,9.938601e+03
320003,1.273963e+02
320004,9.818302e+03
...,...
323774,6.388271e-01
323775,5.183918e-01
323776,1.208795e+01
323777,2.196892e+01


## In-vessel components

In [25]:
rwclinput

WindowsPath('d:/dvp/dev/mcnp/upp/wrk/up08/Radwaste_Checklist_UPP08.xlsx')

In [26]:
rwclinput_df = pd.read_excel(rwclinput, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  # Yes, UPP02 for UPP08
rwclinput_df

,Mass of component (see 4.3)
Equipment part (see 4.1),
DFW Left,1048 kg
DFW Right,1004 kg
DSM,3629 kg
Side plate #1-18 (18 pieces),27 kg each
Frame Top,63 kg
Frame Bottom,138 kg
Frame Blade #1,21 kg
Frame Blade #2,21 kg
Frame Blade #3,19 kg


In [27]:
mass_column = rwclinput_df.columns[0]

In [28]:
def fix_mass_values(row):
    mass_text = row[mass_column]
    mass = float(mass_text.split()[0])
    row[mass_column] = mass

In [29]:
in_vessel_components = rwclinput_df.copy()
in_vessel_components.apply(fix_mass_values, axis=1)
in_vessel_components.rename(columns={mass_column: "Mass, kg"}, inplace=True)
in_vessel_components

,"Mass, kg"
Equipment part (see 4.1),
DFW Left,1048.0
DFW Right,1004.0
DSM,3629.0
Side plate #1-18 (18 pieces),27.0
Frame Top,63.0
Frame Bottom,138.0
Frame Blade #1,21.0
Frame Blade #2,21.0
Frame Blade #3,19.0


In [30]:
in_vessel_components["Activity, Bq"] = np.zeros(len(in_vessel_components.index), dtype=float)
in_vessel_components["H3 Activity, Bq"] = np.zeros(len(in_vessel_components.index), dtype=float)

In [31]:
def collect_cells_map(rwcl_key, inp_key_map, component_to_cell_map) -> tp.Optional[Dict[int, float]]:
    keys_in_cell_map = inp_key_map[rwcl_key]
    if keys_in_cell_map:
        cell_fraction_map = defaultdict(float)
        for k in keys_in_cell_map:
            if "Взять" in k:
                print(f"Warning: use data for UP02 for RWCL id \"{rwcl_key}\"")
            else:
                cells_fraction_map_for_key = component_to_cell_map[k]
                if cells_fraction_map_for_key:
                    for k, v in cells_fraction_map_for_key.items():
                        cell_fraction_map[int(k)] += v
        for k, v in cell_fraction_map.items():
            if 1.0 < v:
                print(f"Warning: ramp down fraction owverflow in cell {k}: {v:.3g}")
                cell_fraction_map[k] = 1.0
        return cell_fraction_map
    else:
        print(f"Warning: cannot find mapping for RWCL id \"{rwcl_key}\"")
        return None 

In [32]:
def compute_activity(cell_fraction_map: Dict[int, float], cell_activity: pd.DataFrame) -> float:
    
    def mapper(pair: tp.Tuple[int, float]) -> float:
        cell, fraction = pair
        assert isinstance(cell, int), f"Integer cell number is expected, {cell} is found"
        try:
            result = cell_activity.loc[cell].activity * fraction
        except KeyError:
            print(f"Warning: cannot find cell {cell} in cell activity data")
            result = 0.0
        return result
    
    return sum(map(mapper, cell_fraction_map.items()))

In [33]:
for rwclid in in_vessel_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_activity_df)
        in_vessel_components.loc[rwclid, ["Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component \"{rwclid}\"")

In [34]:
in_vessel_components

,"Mass, kg","Activity, Bq","H3 Activity, Bq"
Equipment part (see 4.1),,,
DFW Left,1048.0,2.133900e+15,0.0
DFW Right,1004.0,2.001552e+15,0.0
DSM,3629.0,1.043354e+14,0.0
Side plate #1-18 (18 pieces),27.0,0.000000e+00,0.0
Frame Top,63.0,4.352832e+11,0.0
Frame Bottom,138.0,3.995806e+11,0.0
Frame Blade #1,21.0,9.462825e+09,0.0
Frame Blade #2,21.0,6.306935e+09,0.0
Frame Blade #3,19.0,3.741389e+09,0.0


In [35]:
cell_h3_activity_df.rename(columns={"H3 activity":"activity"}, inplace=True)

In [36]:
for rwclid in in_vessel_components.index:
    cell_fraction_map = collect_cells_map(rwclid, inp_keys, component_to_cell)
    if cell_fraction_map is not None:
        activity = compute_activity(cell_fraction_map, cell_h3_activity_df)
        in_vessel_components.loc[rwclid, ["H3 Activity, Bq"]] = activity
    else:
        print(f"Warning: cannot find activity for RWCL component \"{rwclid}\"")

In [37]:
in_vessel_components["Unit Activity, Bq/kg"] = in_vessel_components["Activity, Bq"] / in_vessel_components["Mass, kg"]
in_vessel_components["Unit H3 Activity, Bq/kg"] = in_vessel_components["H3 Activity, Bq"] / in_vessel_components["Mass, kg"]
in_vessel_components

,"Mass, kg","Activity, Bq","H3 Activity, Bq","Unit Activity, Bq/kg","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
DFW Left,1048.0,2.133900e+15,6.706039e+10,2036163906300.962402,63988921.939904
DFW Right,1004.0,2.001552e+15,6.390358e+10,1993578068860.598389,63648986.824002
DSM,3629.0,1.043354e+14,1.606234e+09,28750462812.795368,442610.642875
Side plate #1-18 (18 pieces),27.0,0.000000e+00,0.000000e+00,0.0,0.0
Frame Top,63.0,4.352832e+11,1.270676e+07,6909256698.923156,201694.582352
Frame Bottom,138.0,3.995806e+11,3.700917e+07,2895511822.356476,268182.379411
Frame Blade #1,21.0,9.462825e+09,5.655333e+04,450610707.798165,2693.015832
Frame Blade #2,21.0,6.306935e+09,4.732029e+05,300330229.515634,22533.472649
Frame Blade #3,19.0,3.741389e+09,6.525541e+05,196915215.94241,34344.951739


In [38]:
def create_msg(row):
#     print("Row:", row)
    activity = row["Activity, Bq"]
    unit_activity = row["Unit Activity, Bq/kg"]
    h3_activity = row["H3 Activity, Bq"]
    unit_h3_activity = row["Unit H3 Activity, Bq/kg"]
    if activity > 0.0:
        return f"Activity {activity:.2g} Bq\n({unit_activity:.2g}, Bq/kg),\nH3 Activity {h3_activity:.2g} Bq\n({unit_h3_activity:.2g}, Bq/kg)"
    else:
        return ""

## Interpolate missed values in in-vessel components.

"Generic Screws (50 pieces)", "PP Generics (pads, skids, gripping - 8 pieces)","PP Tubes"  - unit values as for "PP Structure", totals proportional to masses


In [39]:
def update_missed_components(df, refkey, missed_keys):
    unit_activity = df.loc[refkey, ["Unit Activity, Bq/kg"]].values[0]
    unit_h3_activity = df.loc[refkey, ["Unit H3 Activity, Bq/kg"]].values[0]
    df.loc[missed_keys, ["Unit Activity, Bq/kg"]] =  unit_activity
    df.loc[missed_keys, ["Unit H3 Activity, Bq/kg"]] = unit_h3_activity
    masses = df.loc[missed_keys, ["Mass, kg"]].values
    df.loc[missed_keys, ["Activity, Bq"]] = unit_activity * masses
    df.loc[missed_keys, ["H3 Activity, Bq"]] = unit_h3_activity * masses
    return df.loc[missed_keys]

In [40]:
def update_missed_pp_components():
    missed_keys = ["Generic Screws (50 pieces)", "PP Generics (pads, skids, gripping - 8 pieces)","PP Tubes"]
    refkey = "PP structure"
    return update_missed_components(in_vessel_components, refkey, missed_keys)

update_missed_pp_components()

,"Mass, kg","Activity, Bq","H3 Activity, Bq","Unit Activity, Bq/kg","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
Generic Screws (50 pieces),45.0,1.494044e+12,1.157977e+07,33200971984.700928,257328.271145
"PP Generics (pads, skids, gripping - 8 pieces)",199.0,6.606993e+12,5.120833e+07,33200971984.700928,257328.271145
PP Tubes,119.0,3.950916e+12,3.062206e+07,33200971984.700928,257328.271145


In [41]:
in_vessel_components

,"Mass, kg","Activity, Bq","H3 Activity, Bq","Unit Activity, Bq/kg","Unit H3 Activity, Bq/kg"
Equipment part (see 4.1),,,,,
DFW Left,1048.0,2.133900e+15,6.706039e+10,2036163906300.962402,63988921.939904
DFW Right,1004.0,2.001552e+15,6.390358e+10,1993578068860.598389,63648986.824002
DSM,3629.0,1.043354e+14,1.606234e+09,28750462812.795368,442610.642875
Side plate #1-18 (18 pieces),27.0,0.000000e+00,0.000000e+00,0.0,0.0
Frame Top,63.0,4.352832e+11,1.270676e+07,6909256698.923156,201694.582352
Frame Bottom,138.0,3.995806e+11,3.700917e+07,2895511822.356476,268182.379411
Frame Blade #1,21.0,9.462825e+09,5.655333e+04,450610707.798165,2693.015832
Frame Blade #2,21.0,6.306935e+09,4.732029e+05,300330229.515634,22533.472649
Frame Blade #3,19.0,3.741389e+09,6.525541e+05,196915215.94241,34344.951739


In [42]:
in_vessel_components["Radiological data"] = in_vessel_components.apply(create_msg, axis=1)

In [43]:
with pd.ExcelWriter(ROOT / "in-vessel.xlsx", engine="openpyxl", mode="w") as excel:
    in_vessel_components.to_excel(excel, sheet_name="result")

## Inter space components

In [44]:
rwclinput_iss_df = pd.read_excel(rwclinput_iss, sheet_name="UPP02", header=0, usecols=[2,3], index_col=0).iloc[3:]  # Yes, UPP02 for UPP08
rwclinput_iss_df

,Mass of component (see 4.3)
Equipment part (see 4.1),
PC Locking System,118.0
PC Bogies,1812.0
PC Frame,NaN
PC Chasis,1224.0
IS Diagnostic sheet,441.0
IS Frame,245.0
IS Chasis,4800.0
IS Bogies,2420.0
IS Locking System,82.0


In [67]:
iss_components = rwclinput_iss_df.copy()
iss_components.rename(columns={mass_column: "Mass, kg"}, inplace=True)
iss_components

,"Mass, kg"
Equipment part (see 4.1),
PC Locking System,118.0
PC Bogies,1812.0
PC Frame,NaN
PC Chasis,1224.0
IS Diagnostic sheet,441.0
IS Frame,245.0
IS Chasis,4800.0
IS Bogies,2420.0
IS Locking System,82.0


In [72]:
with (ROOT / "component_to_cell_keys.txt").open("w") as stream:
    for key in sorted(component_to_cell.keys()):
        print(key, file=stream)

In [73]:
with (ROOT / "iss_rwcl_keys.txt").open("w") as stream:
    for i in iss_components.index:
        print(i, file=stream)

Ждем mapping от Ромы, продолжить отсюда...

In [105]:
inp1 = set(rwclinput_df.index.values)
inp2 = set(rwclinput_iss_df.index.values)
available = set(djson.keys())

In [106]:
sorted(inp1), sorted(inp2), sorted(available)

(['DFW Left',
  'DFW Right',
  'DSM',
  'DSM Rails (2 pieces)',
  'Frame Blade #1',
  'Frame Blade #2',
  'Frame Blade #3',
  'Frame Blade #4',
  'Frame Blade #5',
  'Frame Blade #6',
  'Frame Blade #7',
  'Frame Blade #8',
  'Frame Blade #9',
  'Frame Bottom',
  'Frame Top',
  'Generic Screws (50 pieces)',
  'Manifold',
  'PP Generics (pads, skids, gripping - 8 pieces)',
  'PP Tubes',
  'PP structure',
  'Rear box shielding (left)',
  'Rear box shielding (right)',
  'Rear box shielding plates (8 pieces)',
  'Rear frame shielding1',
  'Rear frame shielding2',
  'Rear frame shielding3',
  'Rear frame shielding4',
  'Rear frame shielding5',
  'Rear frame shielding6',
  'Rear frame shielding7',
  'Rear frame shielding8',
  'Rear frame shielding9',
  'Side plate #1-18 (18 pieces)'],
 ['Bioshield 1',
  'Bioshield 10',
  'Bioshield 11',
  'Bioshield 12',
  'Bioshield 13',
  'Bioshield 14',
  'Bioshield 15',
  'Bioshield 16',
  'Bioshield 17',
  'Bioshield 18',
  'Bioshield 19',
  'Bioshield 

In [112]:
inp1.intersection(available)

{'DFW Right'}

In [114]:
inp2.intersection(available)

set()

In [42]:
def calc_rwcl_item_activity(cell_activity_df: pd.DataFrame, djson: Dict[str, int]) -> tp.Tuple[Dict[str, float], Dict[str, tp.List[int]]]:
    missed_items_cells = defaultdict(list)
    rwcl_item_activity = defaultdict(float)
    for i, (k, v) in enumerate(djson.items()):
        for c in v:
            if c in cell_activity_df:
                rwcl_item_activity[k] += cell_activity_df.loc[c]
            else:
                missed_items_cells[k].append(c)
    return rwcl_item_activity, missed_items_cells

In [43]:
rwcl_item_activity, missed_items_cells = calc_rwcl_item_activity(cell_activity_df, djson)

In [44]:
rwcl_item_activity

defaultdict(float, {})

## Map RWCL items to activity value

In [46]:
# missed_items_cells

In [185]:
# sorted(rwcl_item_activity.keys())

In [ ]:
# 6                       Side plate #1-18 (18 pieces)
# 17                                    Frame Blade #9
# 18                              DSM Rails (2 pieces)
# 19                                          PP Tubes
# 20    PP Generics (pads, skids, gripping - 8 pieces)
# 22                        Generic Screws (50 pieces)
# 34              Rear box shielding plates (8 pieces)
# 35                                          Manifold
# Name: Equipment part (see 4.1), dtype: object
        
rwcl2json = {
 "Frame Blade #1": 'Frame1',
 "Frame Blade #2": 'Blade2',
 "Frame Blade #3": 'Blade3',
 "Frame Blade #4": 'Blade4',
 "Frame Blade #5": 'Blade5',
 "Frame Blade #6": 'Blade6',
 "Frame Blade #7": 'Blade7',
 "Frame Blade #8": 'Blade8',
 "DFW Left": 'DFW_left',
 "DFW Right": 'DFW_right',
#  'DMS_front',
#  'DMS_rear',
 "DSM": ['DSM','DSM_down', 'DSM_up'],
    
 "PP structure": [
     'Flange',
     'Flange_ext',
     'Trapezoid',
     'Rear_box',
 ]

 "Frame Bottom": 'Frame_bottom',
 "Frame Top": 'Frame_top',
# 'GDC',
 'Interspace_left_block',
 'Interspace_left_frame',
 'Interspace_lower_shielding',
 'Interspace_right_block',
 'Interspace_right_frame',
 'Interspace_top_frame',
 'Mount_bottom',
 'Mount_left',
 'Mount_right',
 'Mount_up',
#  'Reflectometer_left',
#  'Reflectometer_rear',
#  'Reflectometer_right',
 "Rear frame shielding1": 'Shielding1',
#  "Rear frame shielding10": ,
 "Rear frame shielding2": 'Shielding2',
 "Rear frame shielding3": 'Shielding3',
 "Rear frame shielding4": 'Shielding4',
 "Rear frame shielding5": 'Shielding5',
 "Rear frame shielding6": 'Shielding6',
 "Rear frame shielding7": 'Shielding7',
 "Rear frame shielding8": 'Shielding8',
 "Rear frame shielding8": 'Shielding_segment_9',
 "Rear box shielding (left)": 'Shielding_rear_left',
 "Rear box shielding (rightt)": 'Shielding_rear_right',
#  'Vis_exvessel',
#  'Visir_flange',
#  'Visir_invessel',
]

In [186]:
activation_column = np.zeros(len(rwclout_keys), dtype=float)

for i, key in enumerate(rwclout_keys):
    if key is np.NaN:
        activation_column[i] = np.NaN
    else:
        activity = rwcl_item_activity.get(key, None)
        if activity is None:
            print(f"No data for component {i} {key}")
            activation_column[i] = np.NaN
        else:
            activation_column[i] = activity
    
activation_column

No data for component 3 DFW Left
No data for component 4 DFW Right
No data for component 6 Side plate #1-18 (18 pieces)
No data for component 7 Frame Top
No data for component 8 Frame Bottom
No data for component 9 Frame Blade #1
No data for component 10 Frame Blade #2
No data for component 11 Frame Blade #3
No data for component 12 Frame Blade #4
No data for component 13 Frame Blade #5
No data for component 14 Frame Blade #6
No data for component 15 Frame Blade #7
No data for component 16 Frame Blade #8
No data for component 17 Frame Blade #9
No data for component 18 DSM Rails (2 pieces)
No data for component 19 PP Tubes
No data for component 20 PP Generics (pads, skids, gripping - 8 pieces)
No data for component 21 PP structure
No data for component 22 Generic Screws (50 pieces)
No data for component 23 Rear frame shielding1
No data for component 24 Rear frame shielding2
No data for component 25 Rear frame shielding3
No data for component 26 Rear frame shielding4
No data for componen

array([           nan,            nan,            nan,            nan,
                  nan, 9.57321799e+13,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan])

In [170]:
rwclinput_df = pd.read_excel(rwclinput, sheet_name="UPP02", engine="openpyxl")   # Yes, sheet name is "UPP02" in file for UP08 RWCL

In [175]:
rwclout_keys = rwclinput_df[rwclinput_df.columns[2]]

In [176]:
rwclout_keys

0                                                NaN
1                                                NaN
2                                                NaN
3                                           DFW Left
4                                          DFW Right
5                                                DSM
6                       Side plate #1-18 (18 pieces)
7                                          Frame Top
8                                       Frame Bottom
9                                     Frame Blade #1
10                                    Frame Blade #2
11                                    Frame Blade #3
12                                    Frame Blade #4
13                                    Frame Blade #5
14                                    Frame Blade #6
15                                    Frame Blade #7
16                                    Frame Blade #8
17                                    Frame Blade #9
18                              DSM Rails (2 p